In [ ]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
plt.style.use('default')

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

from detection.dataset import VideoDataset, MyConcatDataset, VideoDatasetRNN
from detection.models import TrackNetV2MSE, TrackNetV2NLL, TrackNetV2RNN
from detection.training import train_model
from detection.testing import get_local_maxima

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)

%load_ext autoreload
%autoreload 2

# Trajectory analysis with RANSAC on heatmaps

In [ ]:
from detection.testing import get_local_maxima

# heatmaps_folder = './checkpoints/tracknet_v2_rnn_360_640/phase_3_0/checkpoint_0002_results/heatmaps_test_standard'
heatmaps_folder = './checkpoints/tracknet_v2_mse_360_640/checkpoint_0027_results/heatmaps_test_standard'

N = 300
# center_frame = 100

# frame_indices = np.arange(center_frame-N, center_frame+N+1)
frame_indices = np.arange(2*N+1)

heatmaps = np.zeros((len(frame_indices), 360, 640))
for k_seed, frame_index in enumerate(frame_indices):
    heatmaps[k_seed] = cv2.imread(os.path.join(heatmaps_folder, f"{frame_index}".zfill(6)+'.png'), cv2.IMREAD_GRAYSCALE)/255

cap = cv2.VideoCapture("../datasets/test_standard/video.mp4")
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_indices[0]+5)

frames = np.zeros((len(frame_indices), 360, 640, 3), dtype=np.uint8)
for k_seed, frame_index in enumerate(frame_indices):
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame")
        break
    frames[k_seed] = cv2.cvtColor(cv2.resize(frame, (640, 360)), cv2.COLOR_BGR2RGB)
cap.release()

In [ ]:
max_num_candidates = 20                # maximum supported number of candidates for each frame
n_candidates = np.zeros(len(heatmaps), dtype=int) # number of candidates for each frame

candidates = np.zeros((len(heatmaps), max_num_candidates, 2))  # candidates array
for k_seed in range(len(heatmaps)):
    m = get_local_maxima(heatmaps[k_seed], threshold=0.1, sigma=0.5, include_diagonal=True)
    candidates[k_seed, :len(m)] = m[:max_num_candidates]
    n_candidates[k_seed] = min(len(m), max_num_candidates)

In [ ]:
k_seed = 60
plt.imshow(frames[k_seed])
plt.imshow(heatmaps[k_seed], cmap='gray', alpha=0.7)
plt.scatter(candidates[k_seed][:,1], candidates[k_seed][:,0], s=4, c='r')

plt.show()

## Fit model

In [ ]:
from trajectory_fitting import *

In [ ]:
def show_fit(background, trajectory, candidates, k_seed, k_min, k_mid, k_max, i_seed, i_min, i_mid, i_max):
    plt.imshow(background, zorder=-2)

    # trajectory
    k = np.arange(len(trajectory)) + k_seed - (len(trajectory)-1)//2
    plt.plot(trajectory[k<=k_min,1],trajectory[k<=k_min,0], 'y.-', zorder=-1, alpha=0.2)
    plt.plot(trajectory[k>=k_max,1],trajectory[k>=k_max,0], 'y.-', zorder=-1, alpha=0.2)
    mask = np.logical_and(k>=k_min, k<=k_max)
    plt.plot(trajectory[mask,1],trajectory[mask,0], 'y.-', zorder=-1, alpha=0.8)

    # support and seed
    plt.scatter(candidates[k_min, i_min, 1], candidates[k_min, i_min, 0], c='w', marker='^')
    plt.scatter(candidates[k_mid, i_mid, 1], candidates[k_mid, i_mid, 0], c='w')
    plt.scatter(candidates[k_max, i_max, 1], candidates[k_max, i_max, 0], c='w', marker='s')
    plt.scatter(candidates[k_seed, i_seed, 1], candidates[k_seed, i_seed, 0], c='k', s=5)

    plt.xlim(0, 640)
    plt.ylim(360, 0)
    plt.show()


In [ ]:
# k_seed = 60a
# k_seed = 94

k_min = []
k_max = []
# TODO: check overlap of path supports

info_keys = ['k_seed','k_min','k_mid','k_max','i_seed','i_min','i_mid','i_max','iterations']
l2i = {key: i for i, key in enumerate(info_keys)}

t = time.time()
for k in range(50, 60):
    k_seed = k

    d_threshold = 5
    seed_radius = 20

    N = 30

    parameters, info, trajectories, costs = fit_trajectories(candidates, n_candidates, k_seed, seed_radius, d_threshold, N)
    
    if costs is None:
        print(f"Seed frame: {k}")
        plt.imshow(frames[k])
        plt.show()
        continue


    print(f"Found {len(costs)} seed triplet(s)")
    s = np.argmin(costs)
    print(f"Seed frame: {info[s,l2i['k_seed']]}")
    print(f"Used for fit: {info[s,l2i['k_min']]}, {info[s,l2i['k_mid']]}, {info[s,l2i['k_max']]}")
    print(f"cost = {costs[s]}")
    print(f"Iterations: {info[s,l2i['iterations']]}")
    print(f"v = {parameters[s,0]}")
    print(f"a = {parameters[s,1]}")

    show_fit(frames[info[s,l2i['k_min']]], trajectories[s], candidates,
            *[info[s, l2i[key]] for key in info_keys[:-1]])
    k_min.append(info[s, l2i['k_min']])
    k_max.append(info[s, l2i['k_max']])
t1 = time.time()
print(f"Execution time: {t1-t}")


# Visualize some activations and kernels because why not

In [ ]:
model = TrackNetV2RNN(sequence_length=4)
model.load('checkpoints/tracknet_v2_rnn_360_640/phase_3_0/checkpoint_0002_best.ckpt')
model.eval()
model

In [ ]:
dataset_params = dict(image_size=(360, 640),
                      sequence_length=4,
                      sigma=5,
                      drop_duplicate_frames=False,
                      transform = ToTensor(),
                      target_transform = ToTensor(),
                      grayscale=False)

dataset = VideoDatasetRNN(root="../datasets/prova/", **dataset_params)

In [ ]:
counter = 0

def get_encoding_layer(desired_block=1, subblock=0):
    layers = []
    for i, block in enumerate(model.children()):
        # print(i)
        if i%2 == 1:
            layers.append(block)
        for j, block_element in enumerate(block.children()):
            #print(i, j)
            for k, layer in enumerate(block_element.children()):
                layers.append(layer)
                # print(i, j, k)
                if type(layer) is torch.nn.ReLU and i==2*desired_block and j==subblock:
                    break
            if type(layer) is torch.nn.ReLU and i==2*desired_block and j==subblock:
                break
        if type(layer) is torch.nn.ReLU and i==2*desired_block:
            break
    return layers

def compute_activations(layers, input):
    activation = input.unsqueeze(dim=0)
    with torch.no_grad():
        for l in layers:
            activation = l(activation)

    return activation.squeeze().numpy()

In [ ]:
frames, labels = dataset[50]
frames = frames.to(torch.float32)

In [ ]:
w, h, dpi = 300*2*16/9, 300, 100

fig, axs = plt.subplots(ncols=2, figsize=(w/dpi, h/dpi), dpi=dpi)

axs[0].imshow(frames[-3:].numpy().transpose(1, 2, 0))
axs[0].set_title("Input frame (last in sequence)")

axs[1].imshow(labels[0])
axs[1].set_title("Ground truth")

fig.tight_layout(pad=0.2)
plt.show()

In [ ]:
noise_part = np.linspace(0, 1, 10)
c = []

for n in noise_part:
    with torch.no_grad():
        f = (1-n)*frames + n*torch.randn(frames.shape)
        out = model(f.unsqueeze(dim=0)).squeeze().numpy()
    c.append(out.max())
plt.plot(noise_part, c)

In [ ]:
n = 0.07
with torch.no_grad():
    f = (1-n)*frames + n*torch.randn(frames.shape)
    out = model(f.unsqueeze(dim=0)).squeeze().numpy()
plt.imshow(out)
plt.colorbar()
plt.show()

In [ ]:
frames[:3] = torch.zeros(3, 360, 640)

In [ ]:
block = 2
subblock = 1

activations = compute_activations(get_encoding_layer(block, subblock), frames)
activations.shape

In [ ]:
(dead_activations, ) = np.where(activations.max(axis=(1,2))==0)
print(f"Of {activations.shape[0]} activations, {dead_activations.size} are dead and {activations.shape[0]-dead_activations.size} are not.")

In [ ]:
height_pixels = 1080
top_adjust = 1

w, h, dpi = height_pixels*16/9*top_adjust, height_pixels, 100
fig, axs = plt.subplots(nrows=8, ncols=8, figsize=(w/dpi, h/dpi), dpi=dpi)

i_0 = 0

for k, ax in enumerate(axs.ravel()):
    ax.imshow(activations[k+i_0], cmap='gray')
    # ax.set_title(i)
    ax.set_axis_off()

#fig.suptitle(f"Activations in encoding block {block}, subblock {subblock}")

fig.tight_layout(pad=0.5)
fig.subplots_adjust(top=top_adjust)

#fig.savefig(f"{block}_{subblock}.png")

plt.show()

In [ ]:
model.state_dict().keys()

In [ ]:
dk = 4

kernels = model.state_dict()['vgg_conv1.1.0.weight'].numpy()
biases = model.state_dict()['vgg_conv1.1.0.bias'].numpy()
w, h, dpi = 800, 800, 100
fig, axs = plt.subplots(nrows=8, ncols=8, figsize=(w/dpi, h/dpi), dpi=dpi)

print(kernels.shape)
print(biases[dk])

min_val = kernels[dk].min()
max_val = kernels[dk].max()
print(min_val, max_val)

max_val=max((max_val, -min_val))
min_val=min((-max_val, min_val))

for k, ax in enumerate(axs.ravel()):
    ax.imshow(kernels[dk,k], cmap='RdBu', vmin=min_val, vmax=max_val)
    ax.set_axis_off()

#fig.suptitle(f"Kernel {k}, bias = {biases[k]:.2g}")
fig.tight_layout(pad=0.2)
plt.show()
